In [2]:
import sys
sys.path.append('/data/lixinag/project/GarmentCode/A_RESULT/smplx/SMPL_python_v.1.1.0/smpl')
from smpl_webuser.serialization import load_model
import numpy as np

## Load SMPL model (here we load the female model)
## Make sure path is correct
m = load_model('/data/lixinag/project/GarmentCode/A_RESULT/smplx/SMPL_python_v.1.1.0/smpl/models/basicmodel_f_lbs_10_207_0_v1.1.0.pkl')

def change2_Tpose(m):
    t_pose_angles = np.zeros(72)
    m.pose[:] = t_pose_angles
    return m

def change2_A40pose(m):
    a_pose_angles = np.zeros(72)
    # Convert 40 degrees to radians
    angle_40_rad = np.radians(40)
    # Set shoulder abduction (arms out to sides)
    # Left shoulder abduction (around Y-axis)
    a_pose_angles[44] = angle_40_rad  # Left shoulder abduction
    # Right shoulder abduction (around Y-axis) 
    a_pose_angles[41] = -angle_40_rad  # Right shoulder abduction (negative for opposite side)
    
    # Set the pose parameters
    m.pose[:] = a_pose_angles
    return m

def change_pose_manual(m, pose):
    m.pose[:] = pose
    return m

## Assign random pose and shape parameters
# m.pose[:] = np.random.rand(m.pose.size) * .2
# m = change2_Tpose(m)
# m = change2_A40pose(m)
m = change_pose_manual(m, np.asarray([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10134304314851761, -0.11690745502710342, -0.6783420443534851, -0.0952579528093338, 0.09875380247831345, 0.7042748928070068, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))
# m.betas[:] = np.random.rand(m.betas.size) * .03
m.betas[:] = np.zeros(m.betas.size)

## Write to an .obj file
# outmesh_path = '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110.obj'
# outmesh_path = '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_Tpose.obj'
# outmesh_path = '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_A40pose.obj'
outmesh_path = '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual.obj'
with open( outmesh_path, 'w') as fp:
    for v in m.r:
        fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )

    for f in m.f+1: # Faces are 1-based, not 0-based in obj files
        fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )

## Print message
# print '..Output mesh saved to: ', outmesh_path 
# change to python3 print
print('..Output mesh saved to: ', outmesh_path)

import trimesh
mesh = trimesh.load(outmesh_path)
print(mesh.vertices.shape)
print(mesh.faces.shape)

# show the mesh
mesh.show()

..Output mesh saved to:  /data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual.obj
(6890, 3)
(13776, 3)


In [13]:
# SMPL joint names from blender_smpl.py
SMPL_JOINT_NAMES = {
    0:  'Pelvis',
    1:  'L_Hip',        4:  'L_Knee',            7:  'L_Ankle',           10: 'L_Foot',
    2:  'R_Hip',        5:  'R_Knee',            8:  'R_Ankle',           11: 'R_Foot',
    3:  'Spine1',       6:  'Spine2',            9:  'Spine3',            12: 'Neck',            15: 'Head',
    13: 'L_Collar',     16: 'L_Shoulder',       18: 'L_Elbow',            20: 'L_Wrist',         22: 'L_Hand',
    14: 'R_Collar',     17: 'R_Shoulder',       19: 'R_Elbow',            21: 'R_Wrist',         23: 'R_Hand',
}

def rodrigues_to_mat(rotvec):
    """
    Convert Rodrigues rotation vector to rotation matrix.
    Based on the implementation from blender_smpl.py
    """
    theta = np.linalg.norm(rotvec)
    if theta > 0:
        r = (rotvec/theta).reshape(3, 1)
        r = r.flatten()
    else:
        r = rotvec
    
    cost = np.cos(theta)
    mat = np.asarray([[0, -r[2], r[1]],
                     [r[2], 0, -r[0]],
                     [-r[1], r[0], 0]])
    
    return cost * np.eye(3) + (1-cost) * r.dot(r.T) + np.sin(theta) * mat

def change_shape_betas(model, betas):
    """
    Change SMPL model shape using betas parameter.
    
    Args:
        model: SMPL model object
        betas: numpy array of shape parameters (10 values for basic model)
    
    Returns:
        vertices: Updated vertex positions
        joints: Updated joint positions
    """
    # Set betas (shape parameters)
    model.betas[:] = betas
    
    # Get updated vertices and joints
    vertices = model.r
    joints = model.J
    
    return vertices, joints

def change_pose_theta(model, pose_params):
    """
    Change SMPL model pose using theta parameters.
    
    Args:
        model: SMPL model object
        pose_params: numpy array of pose parameters (72 values for 24 joints * 3)
    
    Returns:
        vertices: Updated vertex positions
        joints: Updated joint positions
    """
    # Set pose parameters
    model.pose[:] = pose_params
    
    # Get updated vertices and joints
    vertices = model.r
    joints = model.J
    
    return vertices, joints

def change_shape_and_pose(model, betas, pose_params):
    """
    Change both shape and pose of SMPL model.
    
    Args:
        model: SMPL model object
        betas: numpy array of shape parameters
        pose_params: numpy array of pose parameters
    
    Returns:
        vertices: Updated vertex positions
        joints: Updated joint positions
    """
    # Set both shape and pose
    model.betas[:] = betas
    model.pose[:] = pose_params
    
    # Get updated vertices and joints
    vertices = model.r
    joints = model.J
    
    return vertices, joints

def get_joint_positions(model):
    """
    Get current joint positions with names.
    
    Args:
        model: SMPL model object
    
    Returns:
        joint_dict: Dictionary mapping joint names to positions
    """
    joints = model.J
    joint_dict = {}
    
    for idx, name in SMPL_JOINT_NAMES.items():
        joint_dict[name] = joints[idx]
    
    return joint_dict

def create_random_shape():
    """
    Create random shape parameters (betas).
    
    Returns:
        betas: Random shape parameters
    """
    return np.random.normal(0.0, 1.0, 10)

def create_random_pose():
    """
    Create random pose parameters (thetas).
    
    Returns:
        pose_params: Random pose parameters
    """
    return np.random.normal(0.0, 0.1, 72)

def reset_to_default_pose(model):
    """
    Reset model to default T-pose.
    
    Args:
        model: SMPL model object
    
    Returns:
        vertices: Default pose vertices
        joints: Default pose joints
    """
    # Reset pose to zero
    model.pose[:] = 0.0
    
    # Get default pose
    vertices = model.r
    joints = model.J
    
    return vertices, joints

def apply_pose_to_joint(model, joint_name, rotation_angles):
    """
    Apply rotation to a specific joint.
    
    Args:
        model: SMPL model object
        joint_name: Name of the joint to rotate
        rotation_angles: [x, y, z] rotation angles in radians
    
    Returns:
        vertices: Updated vertex positions
        joints: Updated joint positions
    """
    # Find joint index
    joint_idx = None
    for idx, name in SMPL_JOINT_NAMES.items():
        if name == joint_name:
            joint_idx = idx
            break
    
    if joint_idx is None:
        raise ValueError(f"Joint name '{joint_name}' not found")
    
    # Convert Euler angles to rotation matrix and then to axis-angle
    from scipy.spatial.transform import Rotation
    
    # Create rotation matrix from Euler angles
    r = Rotation.from_euler('xyz', rotation_angles)
    
    # Convert to axis-angle representation
    axis_angle = r.as_rotvec()
    
    # Set the pose parameters for this joint
    start_idx = joint_idx * 3
    model.pose[start_idx:start_idx+3] = axis_angle
    
    # Get updated model
    vertices = model.r
    joints = model.J
    
    return vertices, joints

# Example usage functions
def example_shape_change():
    """Example of changing model shape"""
    print("Changing model shape...")
    
    # Create random shape parameters
    betas = create_random_shape()
    print(f"Shape parameters (betas): {betas}")
    
    # Apply shape change
    vertices, joints = change_shape_betas(m, betas)
    print(f"Model updated with {len(vertices)} vertices and {len(joints)} joints")
    
    return vertices, joints

def example_pose_change():
    """Example of changing model pose"""
    print("Changing model pose...")
    
    # Create random pose parameters
    pose_params = create_random_pose()
    print(f"Pose parameters (thetas): {pose_params[:10]}...")  # Show first 10 values
    
    # Apply pose change
    vertices, joints = change_pose_theta(m, pose_params)
    print(f"Model updated with {len(vertices)} vertices and {len(joints)} joints")
    
    return vertices, joints

def example_joint_rotation():
    """Example of rotating a specific joint"""
    print("Rotating specific joint...")
    
    # Rotate the right shoulder
    rotation_angles = [0, 0, np.pi/4]  # 45 degrees around Z-axis
    vertices, joints = apply_pose_to_joint(m, 'R_Shoulder', rotation_angles)
    
    print(f"Rotated R_Shoulder by {np.degrees(rotation_angles)} degrees")
    print(f"Model updated with {len(vertices)} vertices and {len(joints)} joints")
    
    return vertices, joints

def get_current_model_info():
    """Get current model information"""
    print("Current model information:")
    print(f"Number of vertices: {len(m.r)}")
    print(f"Number of joints: {len(m.J)}")
    print(f"Shape parameters (betas): {m.betas}")
    print(f"Pose parameters (thetas): {m.pose[:10]}...")  # Show first 10 values
    
    # Show joint names and positions
    joint_dict = get_joint_positions(m)
    print("\nJoint positions:")
    for name, pos in list(joint_dict.items())[:5]:  # Show first 5 joints
        print(f"  {name}: {pos}")
    print("  ...")
    
    return joint_dict

def save_mesh(model, outmesh_path: str = '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual.obj'):
    with open(outmesh_path, 'w') as fp:
        for v in model.r:
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )

        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
    print('..Output mesh saved to: ', outmesh_path)

def display_mesh(outmesh_path: str = '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual.obj'):
    """Display the mesh"""
    import trimesh
    mesh = trimesh.load(outmesh_path)
    mesh.show()

In [12]:
import sys
sys.path.append('/data/lixinag/project/GarmentCode/A_RESULT/smplx/SMPL_python_v.1.1.0/smpl')
from smpl_webuser.serialization import load_model
import numpy as np

# m = load_model('/data/lixinag/project/GarmentCode/A_RESULT/smplx/SMPL_python_v.1.1.0/smpl/models/basicmodel_f_lbs_10_207_0_v1.1.0.pkl')
m = load_model('/data/lixinag/project/GarmentCode/A_RESULT/smplx/SMPL_FEMALE.pkl')

In [14]:
outmesh_path = '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual.obj'
# outmesh_path = '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_Tpose_manual.obj'
save_mesh(m, outmesh_path=outmesh_path)
import trimesh
mesh = trimesh.load(outmesh_path)
mesh.show()

..Output mesh saved to:  /data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual.obj


In [15]:
def change_model_pose(model, joint_name, rotation_angles):
    """
    Change the model's pose by rotating a specific joint.
    
    Args:
        model: SMPL model object
        joint_name: Name of the joint to rotate (e.g., 'R_Shoulder', 'L_Hip', 'Spine1')
        rotation_angles: [x, y, z] rotation angles in radians or degrees
    
    Returns:
        vertices: Updated vertex positions
        joints: Updated joint positions
    """
    # Find joint index from SMPL_JOINT_NAMES
    joint_idx = None
    for idx, name in SMPL_JOINT_NAMES.items():
        if name == joint_name:
            joint_idx = idx
            break
    
    if joint_idx is None:
        raise ValueError(f"Joint name '{joint_name}' not found. Available joints: {list(SMPL_JOINT_NAMES.values())}")
    
    # Convert rotation angles to radians if they're in degrees
    if np.max(np.abs(rotation_angles)) > np.pi:
        rotation_angles = np.radians(rotation_angles)
        print(f"Converted angles to radians: {np.degrees(rotation_angles)} degrees -> {rotation_angles} radians")
    
    # Convert Euler angles to axis-angle representation (Rodrigues vector)
    from scipy.spatial.transform import Rotation
    
    # Create rotation matrix from Euler angles
    r = Rotation.from_euler('xyz', rotation_angles)
    
    # Convert to axis-angle representation (Rodrigues vector)
    axis_angle = r.as_rotvec()
    
    # Set the pose parameters for this joint
    start_idx = joint_idx * 3
    model.pose[start_idx:start_idx+3] = axis_angle

In [17]:
change_model_pose(m, 'R_Shoulder', [0, 0, 40])
change_model_pose(m, 'L_Shoulder', [0, 0, -40])
def save_y_min_greater_than_zero_mesh(model, outmesh_path: str = '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual.obj'):
    y_min = model.r[:, 1].min()
    y_bias = -y_min
    with open( outmesh_path, 'w') as fp:
        for v in model.r:
                fp.write( 'v %.10f %.10f %.10f\n' % ( v[0], v[1] + y_bias, v[2]) )

        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d//%d %d//%d %d//%d\n' %  (f[0], f[0], f[1], f[1], f[2], f[2]) )
    print('..Output mesh saved to: ', outmesh_path)

save_mesh(m, '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual_ori.obj')
# save_mesh(m, '/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_Tpose_manual_ori.obj')
save_y_min_greater_than_zero_mesh(m, outmesh_path=outmesh_path)
import trimesh
mesh = trimesh.load(outmesh_path)
mesh.show()

Converted angles to radians: [ 0.  0. 40.] degrees -> [0.        0.        0.6981317] radians
Converted angles to radians: [  0.   0. -40.] degrees -> [ 0.         0.        -0.6981317] radians
..Output mesh saved to:  /data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual_ori.obj
..Output mesh saved to:  /data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_Tpose_manual.obj


In [18]:
m.pose, m.pose.shape, m.betas

(<chumpy.ch.Ch object at 0x7f955f76f400>
 [ 0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.        -0.6981317  0.         0.         0.6981317
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.       ],
 (72,),
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.])

In [8]:
def get_reference_y_range():
    """获取参考文件f_smpl_average_A40.obj的Y坐标范围"""
    import trimesh
    
    reference_mesh = trimesh.load("/data/lixinag/project/GarmentCode/A_RESULT/smplx/test_smpl/hello_smpl_110_manual.obj")
    y_min = reference_mesh.vertices[:, 1].min()
    y_max = reference_mesh.vertices[:, 1].max()
    
    print(f"参考文件Y坐标范围: {y_min:.6f} 到 {y_max:.6f}")
    return y_min, y_max
get_reference_y_range()

参考文件Y坐标范围: 0.000000 到 1.658390


(0.0, 1.6583903005)